In [144]:
import pretty_midi
import numpy as np
import pickle

num_notes = 128 # Number of pitches in MIDI
num_time_shifts = 100
num_velocities = 32
min_time_shift = 0.01

In [145]:
mid = pretty_midi.PrettyMIDI('musicnet_midis/Beethoven/2314_qt15_2.mid')

In [146]:
mid.instruments[0].notes[:20]

[Note(start=0.000000, end=0.170785, pitch=69, velocity=1),
 Note(start=0.000000, end=0.345203, pitch=56, velocity=50),
 Note(start=0.348837, end=0.678698, pitch=57, velocity=51),
 Note(start=0.682170, end=1.012031, pitch=61, velocity=53),
 Note(start=1.015503, end=1.345364, pitch=61, velocity=55),
 Note(start=1.348836, end=1.678697, pitch=62, velocity=57),
 Note(start=1.682169, end=2.012030, pitch=66, velocity=59),
 Note(start=2.015502, end=2.345363, pitch=58, velocity=61),
 Note(start=2.348835, end=2.678696, pitch=59, velocity=63),
 Note(start=2.682168, end=3.012029, pitch=63, velocity=65),
 Note(start=3.015501, end=3.345362, pitch=63, velocity=67),
 Note(start=3.348834, end=3.678695, pitch=64, velocity=69),
 Note(start=3.682167, end=4.012028, pitch=68, velocity=70),
 Note(start=4.015500, end=4.678694, pitch=73, velocity=60),
 Note(start=4.682166, end=4.845360, pitch=71, velocity=60),
 Note(start=4.848832, end=5.012027, pitch=69, velocity=60),
 Note(start=5.015499, end=5.178693, pitch

In [154]:
data = np.load('preprocessed_data/recording3.npy', allow_pickle=True)
inst = np.load('preprocessed_data/instruments3.npy', allow_pickle=True)

In [155]:
def data_to_midi(data, instrument_number):
    instrument=pretty_midi.Instrument(program=instrument_number,name=pretty_midi.program_to_instrument_name(instrument_number))
    curr_time = 0
    curr_velocity = 0
    #start time and velocity of the current note being played
    note_start = {}
    note_velocity = {}
    for idx in data:
        if idx < num_notes:
            #note on event
            print("note on events")
            if idx in note_start:
                raise ValueError("note" + str(idx) + "has already started!")
            note_start[idx] = curr_time
            note_velocity[idx] = curr_velocity
        elif idx < 2*num_notes:
            #note off event
            print("note off event")
            curr_note = idx - num_notes
            if curr_note not in note_start:
                raise ValueError("note" + str(idx) + "has not been started!")
            note=pretty_midi.Note(velocity=note_velocity[curr_note], pitch=curr_note, start=note_start[curr_note], end=curr_time)
            instrument.notes.append(note)
            del note_start[curr_note]
            del note_velocity[curr_note]
        elif idx < 2*num_notes + num_velocities:
            #velocity event
            #reversing the quantization in preprocess operation
            print("velocity event")
            curr_velocity=(idx-2*num_notes)*128/num_velocities
        elif idx < 2*num_notes + num_time_shifts + num_velocities:
            #time shift event
            print("time shift event")
            curr_time += (idx - 2*num_notes - num_velocities + 1)*min_time_shift
    return instrument
    

In [ ]:
recording = pretty_midi.PrettyMIDI()
for i,j in zip(data, inst):
    recording.instruments.append(data_to_midi(i,j))

In [157]:
recording.instruments[0].notes[:10]
#data[0][0].tolist().index(1)

[Note(start=0.000000, end=0.170000, pitch=69, velocity=0.0),
 Note(start=0.000000, end=0.340000, pitch=56, velocity=48.0),
 Note(start=0.340000, end=0.670000, pitch=57, velocity=48.0),
 Note(start=0.670000, end=1.000000, pitch=61, velocity=52.0),
 Note(start=1.000000, end=1.330000, pitch=61, velocity=52.0),
 Note(start=1.330000, end=1.660000, pitch=62, velocity=56.0),
 Note(start=1.660000, end=1.990000, pitch=66, velocity=56.0),
 Note(start=1.990000, end=2.320000, pitch=58, velocity=60.0),
 Note(start=2.320000, end=2.650000, pitch=59, velocity=60.0),
 Note(start=2.650000, end=2.980000, pitch=63, velocity=64.0)]

In [98]:
file = open( "preprocessed_data_fnames.p", "rb" )
res = pickle.load(file)
file.close()

In [101]:
res

['Haydn/2106_op64n5_3.mid',
 'Haydn/2104_op64n5_1.mid',
 'Haydn/2105_op64n5_2.mid',
 'Beethoven/2314_qt15_2.mid',
 'Beethoven/2483_qt05_4.mid',
 'Beethoven/2522_vcs3_2.mid',
 'Beethoven/2505_op103-2.mid',
 'Beethoven/2532_ps32_01.mid',
 'Beethoven/2335_vns08_2.mid',
 'Beethoven/2404_ps31_01.mid',
 'Beethoven/2608_ps16_03.mid',
 'Beethoven/2556_ps30_02.mid',
 'Beethoven/2595_ps18_03.mid',
 'Beethoven/2315_qt15_3.mid',
 'Beethoven/2486_ps04_02.mid',
 'Beethoven/2381_qt13_1.mid',
 'Beethoven/2471_ps03_02.mid',
 'Beethoven/2405_ps31_02.mid',
 'Beethoven/2491_ps04_03.mid',
 'Beethoven/2366_qt12_2.mid',
 'Beethoven/2424_ps01_03.mid',
 'Beethoven/2423_ps01_02.mid',
 'Beethoven/2415_lvbsext2.mid',
 'Beethoven/2576_ps08_03.mid',
 'Beethoven/2463_lvb23c.mid',
 'Beethoven/2603_ps26_01.mid',
 'Beethoven/2398_op47_2.mid',
 'Beethoven/2509_ps02_01.mid',
 'Beethoven/2497_qt11_4.mid',
 'Beethoven/2555_ps30_01.mid',
 'Beethoven/2538_ps20_02.mid',
 'Beethoven/2528_ps05_02.mid',
 'Beethoven/2444_ps29_04.

In [136]:
res.index('Haydn/2106_op64n5_3.mid')

0